# Raveling: Detecting fraudulent transactions

### Detailed description goes here

In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from tqdm import tqdm
tqdm.pandas()

In [2]:
print(sys.executable)

/Users/viktoria/opt/anaconda3/envs/ravelin/bin/python


Define directories

In [3]:
base = os.path.join('..')
code = os.path.join(base, 'code')
raw = os.path.join(base, 'raw_data')
preproc = os.path.join(base, 'preproc')
viz = os.path.join(base, 'visualisation')
res = os.path.join(base, 'results')

Import the dataset ('raw_data/customers.json')

In [4]:
os.chdir(raw)
customers = pd.DataFrame()
for line in open('customers.json', 'r'):
    customers = customers.append(pd.json_normalize(json.loads(line)))

customers = customers.reset_index(drop=True)

In [5]:
customers.shape

(168, 9)

In [6]:
customers.head()

,fraudulent,orders,paymentMethods,transactions,customer.customerEmail,customer.customerPhone,customer.customerDevice,customer.customerIPAddress,customer.customerBillingAddress
0,False,"[{'orderId': 'vjbdvd', 'orderAmount': 18, 'ord...","[{'paymentMethodId': 'wt07xm68b', 'paymentMeth...","[{'transactionId': 'a9lcj51r', 'orderId': 'vjb...",josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896"
1,True,"[{'orderId': 'nlghpa', 'orderAmount': 45, 'ord...","[{'paymentMethodId': 'y3xp697jx', 'paymentMeth...","[{'transactionId': '5mi94sfw', 'orderId': 'nlg...",evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...
2,False,"[{'orderId': 'yk34y2', 'orderAmount': 33, 'ord...","[{'paymentMethodId': '8pneoi03z', 'paymentMeth...","[{'transactionId': 'q3lyvbza', 'orderId': 'yk3...",andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250"
3,False,"[{'orderId': 'fbz9ep', 'orderAmount': 34, 'ord...","[{'paymentMethodId': 'pdxjdwui4', 'paymentMeth...","[{'transactionId': 'vx4cjc27', 'orderId': 'fbz...",rubenjuarez@yahoo.com,670.664.8168x94985,slovx60t0i558may4ks0,95de:8565:5a66:792c:26e0:6cfb:7d87:11af,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM..."
4,True,"[{'orderId': '56h7iw', 'orderAmount': 71, 'ord...","[{'paymentMethodId': 'w1i1zq3rg', 'paymentMeth...","[{'transactionId': 'q8j3dgni', 'orderId': '56h...",uchen@malone.com,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,196.89.235.192,"148 Russell Lodge Apt. 445\nPort Jenniferside,..."


### 1. Exploring variables: determine the units of observation

In [7]:
len(customers['customer.customerEmail'].unique())

161

In [8]:
[(c, type(customers.loc[0,c])) for c in customers.columns]

[('fraudulent', numpy.bool_),
 ('orders', list),
 ('paymentMethods', list),
 ('transactions', list),
 ('customer.customerEmail', str),
 ('customer.customerPhone', str),
 ('customer.customerDevice', str),
 ('customer.customerIPAddress', str),
 ('customer.customerBillingAddress', str)]

In [9]:
print('Fraud: ', len(customers[customers['fraudulent']==True]), '\nNon-fraud: ', len(customers[customers['fraudulent']==False]))

Fraud:  61 
Non-fraud:  107


Find out how the variables relate to one another. Dictionary keys must be linked.

In [10]:
print(customers.loc[0, 'orders'][0].keys())

dict_keys(['orderId', 'orderAmount', 'orderState', 'orderShippingAddress'])


In [11]:
print(customers.loc[0, 'transactions'][0].keys())

dict_keys(['transactionId', 'orderId', 'paymentMethodId', 'transactionAmount', 'transactionFailed'])


In [12]:
print(customers.loc[0, 'paymentMethods'][0].keys())

dict_keys(['paymentMethodId', 'paymentMethodRegistrationFailure', 'paymentMethodType', 'paymentMethodProvider', 'paymentMethodIssuer'])


Each order is related to a transaction through 'orderId'. Each payment method is related to a transaction through paymentMethodId.

The units of observation shall be transactions. Create a dataframe where each transaction is a row.

In [13]:
transactions = pd.DataFrame()

for row,val in tqdm(customers.iterrows()):
    
    transaction_list = customers.loc[row, 'transactions']
    order_list = customers.loc[row, 'orders']
    payment_list = customers.loc[row, 'paymentMethods']
    
    #each transaction will be a row - find the corresponding info
    for t in transaction_list:
        
        cust_info = pd.DataFrame(val).T.reset_index(drop=True)
        trans_info = pd.DataFrame.from_dict({k:v for k,v in t.items()}, orient='index').T
       
    
        #identify the corresponding order
        try:
            this_order = [o for o in order_list if o.get('orderId')==t.get('orderId')][0]
            order_info = pd.DataFrame.from_dict({k:v for k,v in this_order.items()}, orient='index').T
        except:
            order_info = pd.DataFrame(data={'orderId':[None], 
                                            'orderAmount':[None], 
                                            'orderState':[None], 
                                            'orderShippingAddress':[None]})
    
        
        #identify the corresponding payment info
        try:
            this_payment = [p for p in payment_list if p.get('paymentMethodId')==t.get('paymentMethodId')][0]
            payment_info = pd.DataFrame.from_dict({k:v for k,v in this_payment.items()}, orient='index').T
        except:
            payment_info = pd.DataFrame(data={'paymentMethodId':[None], 
                                              'paymentMethodRegistrationFailure':[None], 
                                              'paymentMethodType':[None], 
                                              'paymentMethodProvider':[None], 
                                              'paymentMethodIssuer':None})
        
            
        #join them
        newrow = pd.concat([cust_info, trans_info, order_info, payment_info], axis=1)
        newrow = newrow.loc[:, ~newrow.columns.duplicated()]
        transactions = transactions.append(newrow).reset_index(drop=True)
        
        
transactions = transactions.drop(['orders', 'paymentMethods', 'transactions', 'orderAmount'], axis=1)

168it [00:01, 159.44it/s]


In [14]:
transactions.head()

,fraudulent,customer.customerEmail,customer.customerPhone,customer.customerDevice,customer.customerIPAddress,customer.customerBillingAddress,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed,orderState,orderShippingAddress,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer
0,False,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",a9lcj51r,vjbdvd,wt07xm68b,18,False,pending,"5493 Jones Islands\nBrownside, CA 51896",True,card,JCB 16 digit,Citizens First Banks
1,False,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",y4wcv03i,yp6x27,wt07xm68b,26,False,fulfilled,"5493 Jones Islands\nBrownside, CA 51896",True,card,JCB 16 digit,Citizens First Banks
2,True,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,5mi94sfw,nlghpa,41ug157xz,45,False,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497",False,bitcoin,American Express,Bastion Banks
3,True,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,br8ba1nu,uw0eeb,41ug157xz,23,False,fulfilled,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,False,bitcoin,American Express,Bastion Banks
4,True,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,a33145ss,bn44oh,y3xp697jx,43,True,fulfilled,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",True,bitcoin,VISA 16 digit,Solace Banks
